In [1]:
import gensim
from gensim import utils
import numpy as np
import sys
from sklearn.datasets import fetch_20newsgroups
from nltk import word_tokenize
from nltk import download
from nltk.corpus import stopwords
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
%matplotlib inline

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import glob
import string
import glob
from tqdm import tqdm 
#import pdfminer
from pdfminer.high_level import extract_text
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk 
nltk.download('stopwords')
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
import nltk
nltk.download('punkt')
from nltk import sent_tokenize
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
from nltk.probability import FreqDist

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/sunilmcesh/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/sunilmcesh/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/sunilmcesh/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [3]:
PAPER_OF_INTEREST_FNAME  = glob.glob('/Volumes/Seagate Backup Plus Drive/JOSS Project/joss-papers-master/*/*/*.pdf')

In [50]:
K = 26
print(PAPER_OF_INTEREST_FNAME[K])

/Volumes/Seagate Backup Plus Drive/JOSS Project/joss-papers-master/joss-papers-master/joss.00044/10.21105.joss.00044.pdf


In [54]:
Paper_interest = PAPER_OF_INTEREST_FNAME[K]

In [55]:
POI_PDF = [extract_text(Paper_interest)]
text = str(POI_PDF)

In [56]:
text

"['DOI: 10.21105/joss.03447\\n\\nSoftware\\n\\n• Review\\n• Repository\\n• Archive\\n\\nEditor: Jacob Schreiber\\n\\nReviewers:\\n\\n• @pps121\\n• @amtseng\\n\\nSubmitted: 16 June 2021\\nPublished: 30 September 2021\\n\\nLicense\\nAuthors of papers retain\\ncopyright and release the work\\nunder a Creative Commons\\nAttribution 4.0 International\\nLicense (CC BY 4.0).\\n\\nCOVID-19 Lung Segmentation\\n\\nRiccardo Biondi∗1, Nico Curti†2, Enrico Giampieri2, and Gastone\\nCastellani1\\n\\n1 Department of Experimental, Diagnostic and Specialty Medicine of Bologna University 2\\neDIMESLab, Department of Experimental, Diagnostic and Specialty Medicine of Bologna University\\n\\nSummary\\n\\nThe COVID-19 Lung Segmentation project provides a novel, unsupervised and fully auto-\\nmated pipeline for the semantic segmentation of ground-glass opacity (GGO) areas in chest\\nComputer Tomography (CT) scans of patients affected by COVID-19.\\nIn the project we\\nprovide a series of scripts and functio

In [ ]:
text = text.replace('-\\n','')
text = text.replace('-\n','')
text = text.replace('\\n',' ')
text = text.replace('\n', ' ')

In [57]:
text

"['DOI: 10.21105/joss.03447\\n\\nSoftware\\n\\n• Review\\n• Repository\\n• Archive\\n\\nEditor: Jacob Schreiber\\n\\nReviewers:\\n\\n• @pps121\\n• @amtseng\\n\\nSubmitted: 16 June 2021\\nPublished: 30 September 2021\\n\\nLicense\\nAuthors of papers retain\\ncopyright and release the work\\nunder a Creative Commons\\nAttribution 4.0 International\\nLicense (CC BY 4.0).\\n\\nCOVID-19 Lung Segmentation\\n\\nRiccardo Biondi∗1, Nico Curti†2, Enrico Giampieri2, and Gastone\\nCastellani1\\n\\n1 Department of Experimental, Diagnostic and Specialty Medicine of Bologna University 2\\neDIMESLab, Department of Experimental, Diagnostic and Specialty Medicine of Bologna University\\n\\nSummary\\n\\nThe COVID-19 Lung Segmentation project provides a novel, unsupervised and fully auto-\\nmated pipeline for the semantic segmentation of ground-glass opacity (GGO) areas in chest\\nComputer Tomography (CT) scans of patients affected by COVID-19.\\nIn the project we\\nprovide a series of scripts and functio

In [9]:
model = gensim.models.KeyedVectors.load_word2vec_format('/Volumes/Seagate Backup Plus Drive/JOSS Project/GoogleNews-vectors-negative300.bin', binary=True)

In [58]:
download('punkt') #tokenizer, run once
download('stopwords') #stopwords dictionary, run once
stop_words = stopwords.words('english')

def preprocess(text):
    text = text.lower()
    doc = word_tokenize(text)
    doc = [word for word in doc if word not in stop_words]
    doc = [word for word in doc if word.isalpha()] #restricts string to alphabetic characters only
    return doc

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/sunilmcesh/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/sunilmcesh/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [59]:
texts = (POI_PDF)

corpus = [preprocess(text) for text in texts]

In [60]:
corpus[0][0]

'doi'

In [61]:
def filter_docs(corpus, texts,  condition_on_doc):
    """
    Filter corpus, texts and labels given the function condition_on_doc which takes
    a doc.
    The document doc is kept if condition_on_doc(doc) is true.
    """
    number_of_docs = len(corpus)

    if texts is not None:
        texts = [text for (text, doc) in zip(texts, corpus)
                 if condition_on_doc(doc)]

    #labels = [i for (i, doc) in zip(labels, corpus) if condition_on_doc(doc)]
    corpus = [doc for doc in corpus if condition_on_doc(doc)]

    print("{} docs removed".format(number_of_docs - len(corpus)))

    return (corpus, texts)

In [62]:
corpus, texts = filter_docs(corpus, texts,  lambda doc: (len(doc) != 0))

0 docs removed


In [63]:
def document_vector(word2vec_model, doc):
    # remove out-of-vocabulary words
    doc = [word for word in doc if word in word2vec_model.vocab]
    return np.mean(word2vec_model[doc], axis=0)

In [64]:
def has_vector_representation(word2vec_model, doc):
    """check if at least one word of the document is in the
    word2vec dictionary"""
    return not all(word not in word2vec_model.vocab for word in doc)

In [65]:
corpus, texts = filter_docs(corpus, texts,  lambda doc: has_vector_representation(model, doc))

0 docs removed


In [66]:
x =[]
for doc in corpus[0]: #look up each doc in model
    x.append(document_vector(model, doc))

In [67]:
X = np.array(x) #list to array

In [68]:
doc

'https'

In [69]:
np.unique(np.array(corpus[0]))

array(['able', 'accuracy', 'achieve', 'achieved', 'acknowledge',
       'acknowledgments', 'acute', 'affected', 'ai', 'aimed', 'al',
       'algorithm', 'already', 'also', 'amount', 'amtseng', 'analysis',
       'analysts', 'another', 'applied', 'approach', 'approaches',
       'approximate', 'archive', 'areas', 'assisted', 'attard',
       'attribution', 'augment', 'author', 'authors', 'automated',
       'automatic', 'available', 'average', 'azienda', 'badia',
       'bartoletti', 'based', 'bash', 'beginning', 'benedittis', 'best',
       'biondi', 'blown', 'bologna', 'bortolani', 'bussini', 'campoli',
       'castellani', 'cattabriga', 'cc', 'center', 'cercenelli', 'chen',
       'cheng', 'chest', 'ciccarese', 'classification', 'clinical',
       'clinically', 'clustering', 'cnn', 'cocozza', 'coefficient',
       'cohort', 'collected', 'color', 'combine', 'commons', 'comparable',
       'computer', 'condition', 'considerably', 'consolidation',
       'contagious', 'convolutional', '

In [70]:
x2 =[]
for doc in np.unique(np.array(corpus[0])): #look up each doc in model
    x2.append(document_vector(model, doc))

In [71]:
X2 = np.array(x2) #list to array

In [72]:
average_word2vec_vector2 = X2.mean(axis=0)

In [73]:
model.similar_by_vector(average_word2vec_vector2)

[('h', 0.7675735950469971),
 ('o', 0.7509527802467346),
 ('f', 0.7381659150123596),
 ('b', 0.7305057644844055),
 ('r', 0.7286273837089539),
 ('¬', 0.7181587815284729),
 ('ts', 0.7167741060256958),
 ('d', 0.7157739400863647),
 ('ar', 0.7124439477920532),
 ('i', 0.7066262364387512)]

In [74]:
X2.shape

(389, 300)

In [75]:
X.shape

(605, 300)

In [76]:
X.mean(axis=0).shape

(300,)

In [77]:
average_word2vec_vector = X.mean(axis=0)

In [78]:
model.similar_by_vector(average_word2vec_vector)

[('h', 0.7642236948013306),
 ('o', 0.7457115054130554),
 ('f', 0.7363426685333252),
 ('b', 0.726196825504303),
 ('ts', 0.7221477627754211),
 ('r', 0.7220396399497986),
 ('¬', 0.7184414863586426),
 ('d', 0.7125791311264038),
 ('t', 0.7109199166297913),
 ('ar', 0.7107546329498291)]

In [37]:
model.similar_by_vector(X[20])

[('h', 0.778066098690033),
 ('o', 0.7700537443161011),
 ('t', 0.7625486254692078),
 ('r', 0.7494496703147888),
 ('ts', 0.7428569197654724),
 ('u', 0.7180385589599609),
 ('¬', 0.7121788859367371),
 ('f', 0.7076941728591919),
 ('cks', 0.7059786319732666),
 ('ta', 0.698557436466217)]

In [35]:
model.most_similar('Journal', topn=20)

[('theJournal', 0.6163490414619446),
 ('theAmerican_Journal', 0.5941680669784546),
 ('Annals', 0.5925253629684448),
 ('Jounal', 0.5823094248771667),
 ('journal', 0.5741861462593079),
 ('journal_Annals', 0.5684102773666382),
 ('PATRICK_JACKSON', 0.5683403015136719),
 ('Physiology_Gastrointestinal', 0.5631081461906433),
 ('By_LULADEY_B.', 0.5580130219459534),
 ('Roentgenology_AJR', 0.557891845703125),
 ('Gazette', 0.5572773814201355),
 ('TADESSE', 0.5548046231269836),
 ('•_GANNETT_STAFF_WRITER', 0.5463052988052368),
 ('currier@sturgisjournal.com', 0.545045793056488),
 ('Herald', 0.5421846508979797),
 ('journal_Archives', 0.5398844480514526),
 ('journal_Cell_Metabolism', 0.5383172035217285),
 ('Bulletin', 0.5375578999519348),
 ('ESTEBAN_PARRA', 0.5362104177474976),
 ('Clinical_Nutrition_Vol', 0.5346276164054871)]